In [1]:
import sys
print(sys.executable)

/Users/daegeun/opt/anaconda3/envs/napy/bin/python


In [2]:
print(sys.path)

['/Users/daegeun/Documents/spaceflight', '/Users/daegeun/opt/anaconda3/envs/napy/lib/python310.zip', '/Users/daegeun/opt/anaconda3/envs/napy/lib/python3.10', '/Users/daegeun/opt/anaconda3/envs/napy/lib/python3.10/lib-dynload', '', '/Users/daegeun/opt/anaconda3/envs/napy/lib/python3.10/site-packages', '/Users/daegeun/opt/anaconda3/envs/napy/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-macosx-10.9-x86_64.egg']


In [21]:
import jax
import jax.numpy as jnp
from jax.numpy import sin, cos
from jax.numpy import pi

In [4]:
x = jnp.arange( 10 )

print( x )

[0 1 2 3 4 5 6 7 8 9]


In [5]:
x

DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [6]:
long_vector = jnp.arange( int( 1e7 ) )

%timeit jnp.dot( long_vector, long_vector )

7.13 ms ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [81]:
def sum_of_squares( x ):
    return jnp.sum( x ** 2 )

In [82]:
sum_of_squares_dx = jax.grad( sum_of_squares )

x = jnp.asarray( [ 1.0, 2.0, 3.0, 4.0 ] )

print( sum_of_squares( x ) )

print( sum_of_squares_dx( x ) )

30.0
[2. 4. 6. 8.]


In [83]:
def sum_squared_error( x, y ):
    return jnp.sum( (x - y) ** 2 )

In [84]:
sum_squared_error_dx = jax.grad( sum_squared_error )

y = jnp.asarray( [1.1, 2.1, 3.1, 4.1] )

print( sum_squared_error_dx( x, y ) )

[-0.20000005 -0.19999981 -0.19999981 -0.19999981]


In [85]:
print( sum_squared_error_dx( y, x ) )

[0.20000005 0.19999981 0.19999981 0.19999981]


In [86]:
jax.grad(sum_squared_error, argnums=(0, 1))( x, y )

(DeviceArray([-0.20000005, -0.19999981, -0.19999981, -0.19999981], dtype=float32),
 DeviceArray([0.20000005, 0.19999981, 0.19999981, 0.19999981], dtype=float32))

In [12]:
import jaxopt

In [13]:
from jaxopt import LevenbergMarquardt

In [14]:
T = 16.194164291066627 
q = -0.17308308096941216

In [91]:
out = jnp.array( [0, 0], dtype=jnp.float32 )

def func( x, q ):
    
    a, b = x
    
    O1 = out.at[0].set( sin( b / 2 ) - q * sin( a / 2 ) )
    O2 = O1.at[1].set( T * ( sin( a / 2 ) ** 3 ) - ( a - b - sin( a ) + sin( b ) ) )
    
    return O2

In [92]:
init_params = jnp.array( [pi, 0] )

solution = LevenbergMarquardt( func ).run( init_params, -0.173 ).params

solution

DeviceArray([ 4.7135043 , -0.24458043], dtype=float32)

In [59]:
def solution( y ):
    
    LM = LevenbergMarquardt( func )
    sl = LM.run( init_params, y ).params
    
    return sl
    
print( jax.jacobian( solution )( -0.173 ) )

[-4.8518003e-04  1.4245435e+00]


In [54]:
len( init_values )

2

In [60]:
solution = LevenbergMarquardt( func ).run( init_params, -0.174 ).params

solution

DeviceArray([ 4.7135    , -0.24600174], dtype=float32)

In [93]:
from jax.numpy import exp 


def func( X, t ):
    
    x, y = X
    
    O1 = out.at[0].set( exp( y+t ) - x )
    O2 = O1.at[1].set( exp( x+t ) + y )
    
    return O2

In [96]:
from time import process_time

In [97]:
init_params = jnp.array( [0.0, 0.0] )

t1 = process_time()

for i in range( 100 ):
    solution = LevenbergMarquardt( func ).run( init_params, 1 ).params

t2 = process_time()

print( t2 - t1 )

solution

87.77253000000002


DeviceArray([ 0.1249296, -3.0799994], dtype=float32)

In [77]:
x, y = solution

In [78]:
pxpt = ( 1 - exp( x+1 ) ) / ( 1/x + exp( x+1 ) )
pypt = ( 1 + exp( y+1 ) ) / ( 1/y - exp( y+1 ) )

In [79]:
def solution( t ):
    
    LM = LevenbergMarquardt( func )
    sl = LM.run( init_params, t ).params
    
    return sl

print( jax.jacobian( solution )( 1.0 ) )

[-0.18764925 -2.5020378 ]


In [80]:
print( pxpt, pypt )

-0.18764928 -2.5020375


In [107]:
A = jnp.array( [1.0, 2.0, 3.0] )
B = jnp.array( [0.0, 0.0, 0.0] )

In [108]:
from jax.numpy import cross

In [109]:
def func( t ):
    
    B1 = B.at[0].set( t )
    B2 = B1.at[1].set( t )
    B3 = B2.at[2].set( t )
    
    return B3 

In [110]:
def function( t ):
    
    D = func( t )
    
    C = cross( A, D )
    
    return C

In [113]:
function_dx = jax.jacobian( function )

In [114]:
function_dx( 1.0 )

DeviceArray([-1.,  2., -1.], dtype=float32, weak_type=True)

In [126]:
def func( x ):
    
    if ( x > 0 ):
        return x
    if ( x < 0 ):
        return 0.0

In [127]:
func_dx = jax.grad( func )

In [128]:
func_dx( 1.0 )

DeviceArray(1., dtype=float32, weak_type=True)

In [129]:
func_dx( 2.0 )

DeviceArray(1., dtype=float32, weak_type=True)

In [130]:
func_dx( -1.0 )

DeviceArray(0., dtype=float32, weak_type=True)